In [1]:
## imports
from qiskit import *
from qiskit.algorithms import *
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit import quantum_info, IBMQ, Aer
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
# backend = BasicAer.get_backend("statevector_simulator")
# quantum_instance = QuantumInstance(backend)
from qiskit.algorithms import AmplitudeEstimation
#from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.visualization import plot_histogram, plot_state_qsphere, plot_bloch_multivector, plot_bloch_vector
import numpy as np
from numpy import pi
import seaborn as sns
from IPython.display import Image
import matplotlib.pyplot as plt
import math


In [2]:
# setup simulator using IBM_id
# IBMQ.save_account("1c62e8a0d2b058d0e797db9c811bc34582df5553e25812301cd1518662c0ab68d2378ac6c197b65f8be944d04b7e7439f034e3926a44fa8088538b3d13782c1a", overwrite= True)
provider = IBMQ.load_account()
IBMQ.get_provider(hub='ibm-q-education', group='iit-madras-1', project='quantum-computin')
# setup required backends 
lima = provider.get_backend('ibmq_lima')
manila = provider.get_backend('ibmq_manila')
qsm = Aer.get_backend('qasm_simulator')
stv = Aer.get_backend('statevector_simulator')
aer = Aer.get_backend('aer_simulator')


##### Grover Operator Setup

In [3]:
## sub-routines for Grover Search ~
def to_oracle(pattern, name= 'oracle'):
    """ Convert a given pattern to an oracle
        input: pattern= a numpy vector with binarry entries 
        output: oracle.Gate    """

    l = len(pattern)
    qr = QuantumRegister(l, name='reg')
    a = AncillaRegister(1, name='ancilla')
    qc = QuantumCircuit(qr, a, name= name+str(pattern))
    for q in range(l):
        if(pattern[q]==0): qc.x(qr[q])
    qc.x(a)
    qc.h(a)
    qc.mcx(qr, a)
    qc.h(a)
    qc.x(a)
    for q in range(l):
        if(pattern[q]==0): qc.x(qr[q])
    #qc.barrier()
    return qc.to_gate()

def diffuser(l):
    """ Gnerate the Diffuser operator for the case where the initial state  is 
        the equal superposition state of all basis vectors 
        input: l= no. of qubits
        output: diffuser.Gate    """
    qr = QuantumRegister(l, name='reg')
    a = AncillaRegister(1, name='ancilla')
    circuit = QuantumCircuit(qr, a, name= 'Diff.')
    
    circuit.h(qr)
    circuit.x(qr)
    
    circuit.x(a)
    circuit.h(a)
    circuit.mcx(qr ,a)
    circuit.h(a)
    circuit.x(a)

    circuit.x(qr)
    circuit.h(qr)
          
    return circuit.to_gate()

def grover_iterate(qc, oracles, diffuser, qreg_u, ancilla, steps):
    """ Run full Grover iteration for given number of steps.
        input:
        qc: QuantumCiruit to append to 
        oracles: a list of oracles generated from 'to_oracle()' function 
        diffuser: a diffuser from 'diffuser()' function 
        steps: no. of grover iterates"""
    for step in range(steps):
        for oracle in oracles:
            qc.append(oracle, list(range(qc.num_qubits)) )
        qc.append(diffuser, list([q for q in qreg_u])+ list(ancilla) )
        # qc.barrier()
    return qc

## modified sub-routine for grover ~
## Based on the 'grover_iterate()' function defined earlier
def grover(patterns, grover_steps):
    
    dim = len(patterns[0])
    
    # create oracles ~\
    oracles = []
    for pattern in patterns : oracles.append( to_oracle(pattern)) 
    
    # create diffuser ~\
    diff = diffuser(dim)

    # create circuit ~\
    qreg = QuantumRegister(dim, name= 'init')
    ancilla = AncillaRegister(1, name='ancilla')
    qc = QuantumCircuit(qreg, ancilla, name='grover'+'^'+str(grover_steps))
    qc = grover_iterate(qc, oracles, diff, qreg, ancilla,grover_steps)
  
    return qc

#### Iterative Quantum Amplitude Estimation 

In [4]:
def s_psi0(p):
    """ Prepare a QuantumCircuit that intiates a state required
        input:
            p= amplitude 
        output:
            s_psi0 gate                                            """
 
    qc = QuantumCircuit(1, name= " S_psi0 ")
    theta = 2*np.arcsin(np.sqrt(p))
    qc.ry(theta, 0)

    return qc

def Q(p, power):
    """ Prepare an Gate to implement 'Q^power' operator
        input:
            p= amplitude
            power= no.of times 'Q' is imposed
        output:
            Q gate"""
    theta = 2*np.arcsin(np.sqrt(p))
    qc = QuantumCircuit(1, name= ' Q'+ '^'+ str(power) )
    qc.ry(2*theta*power, 0)

    return qc

def good_val_measure_single_qubit(p, m, Nshots=1024):
    # p = 0.23
    qreg = QuantumRegister(1, name='qreg')
    creg = ClassicalRegister(1, name= 'creg')
    qc = QuantumCircuit(qreg, creg)

    qc.compose(s_psi0(p).to_gate(), inplace= True)
    qc.compose(Q(p, m).to_gate(), inplace= True)

    qc.measure(qreg, creg)
    
    counts =  execute(qc, backend= qsm, shots= Nshots).result().get_counts()
    if len(counts) == 1:
        if list(counts.keys()) == ['1']: counts['0'] = 0
        else : counts['1'] = 0
    
    return counts['1']/Nshots

In [14]:
## finding next k sub routine
def find_next_k(k_i, theta_l, theta_u, up: bool, r= 2 ):
    K_i= 4*k_i + 2
    theta_max = K_i*theta_l
    theta_min = K_i*theta_u
    K_max = int(0.5/(theta_u - theta_l))
    K = K_max - np.mod((K_max - 2), 4)
    # print('K :', K, 'K_i :', K_i) ## set probe

    ## searching for higher k vals
    while K >= r*K_i :
        q= K/K_i
        # print('K :', K, 'K_i :', K_i) ## set probe
        # print('K/K_i :', q) ## set probe

        theta_min = K * theta_l - int(K * theta_l)
        theta_max = K * theta_u - int(K * theta_u)

        if theta_min <= theta_max <= 0.5 and theta_min <= 0.5 :
            up_u = True
            return int((K - 2) / 4), up_u
        
        if theta_max >= 0.5 and theta_max >= theta_min >= 0.5 :
            up_u = False
            return int((K - 2) / 4), up_u



        # if np.mod(q*theta_max, 2*pi)< pi and np.mod( q*theta_min, 2*pi )< pi :
        #     print('c_1') ## set probe

        #     K_u = K
        #     up_u= True
        #     k_u = (K_u -2)/4
        #     return (k_u, up_u)
        
        # if np.mod(q*theta_max, 2*pi)>= pi and np.mod(q*theta_min, 2*pi)>= pi :
        #     print('c_2') ## set probe
            
        #     K_u= K
        #     up_u = False
        #     k_u = (K_u -2)/4
        #     return (k_u, up_u)
    
        K = K-4
    
    return (k_i, up) 

In [15]:
find_next_k(0, 0, 0.25, True, r =2)

(0, True)

In [16]:
# ## Iterative QAE main 
# def IQAE_single_qubit(p, eps, alpha, Nshots, conf= None ):
#     i = 0 
#     k = [0]
#     K = [0]
#     up = [True]
#     theta_low, theta_up = 0, pi/2
#     T = math.ceil( np.log2(pi/(8*eps))   )
#     L_max = np.arcsin( ((2/Nshots) * np.log(2*T/alpha))**(1//4)  )  ### re-check
#     while (theta_low - theta_up) > 2*eps :
#         i = i+1
#         k_u, up_u = find_next_k(k[i-1], theta_low, theta_up, up[i-1] )
#         K_u = 4*k_u + 2 

#         k.append(k_u)
#         up.append(up_u)
#         K.append(K_u)

#         if K_u > math.ceil(L_max/eps):
#             N = math.cel(Nshots*L_max/eps/K_u/10)
#         else :
#             N = Nshots

#         ## get p(|1>) from qiskit
#         a_u = good_val_measure_single_qubit(p, k_u, Nshots= N)

#         if k_u == k[i-1]:
#             #### comibne all previous results
#             pass
        
#         ## calculate bound
#         eps_u = np.sqrt(  (1/2*N)* np.log((2*T/alpha))     )
#         a_u_max = min(1, a_u + eps_u)
#         a_u_min = max(0, a_u - eps_u)

#         theta = [ np.arccos(1- 2*a_u_max)/K_u, np.arccos(1- 2*a_u_min)/K_u     ]
#         theta_u_min = min(theta)
#         theta_u_max = max(theta) 

#         ## calculate confidence interval
#         theta_low = ( np.mod(np.floor(K_u*theta_low), 2*pi) + theta_u_min )/K_u
#         theta_up = ( np.mod(np.floor(K_u*theta_up), 2*pi) + theta_u_max )/K_u

#     a_low = (np.sin(theta_low))**2
#     a_up = (np.sin(theta_up))**2

#     return ( a_low, a_up)

##### Single Qubit results

In [17]:
################
# p = 0.75
# eps= 10**(-2)
# alpha = 0.5
# Nshots= 100
################


def IQAE_single_qubit(p, eps= 10**(-2), alpha = 0.5, Nshots= 100 ):
    ################
    i = 0 
    k = [0]
    K = [0]
    up = [True]
    theta_low, theta_up = 0 , 1/4
    theta_lows, theta_ups = [], []

    T = math.ceil( np.log2(pi/(8*eps))   )
    L_max = np.arcsin( ((2/Nshots) * np.log(2*T/alpha))**(1//4)  )

    while (theta_up - theta_low) > eps/pi :
        i = i+1
        k_u, up_u = find_next_k(k[i-1], theta_low, theta_up, up[i-1] )
        K_u = 4*k_u + 2 

        ## set probe
        # print("k: ", k_u, 'up: ', up_u)

        k.append(k_u)
        up.append(up_u)
        K.append(K_u)

        if K_u > math.ceil(L_max/eps):
            N = math.cel(Nshots*L_max/eps/K_u/10)
        else :
            N = Nshots

        ## get p(|1>) from qiskit
        a_u = good_val_measure_single_qubit(p, k_u, Nshots= N)
        # print('a_u :', a_u) ## set probe

        if k_u == k[i-1]:
            #### comibne all previous results
            pass
        
        ## calculate bound
        eps_u = np.sqrt(  1/(2*N) * np.log((2*T/alpha))     )
        a_u_max = min(1, a_u + eps_u)
        a_u_min = max(0, a_u - eps_u)
        # print('a_u_max: ', a_u_max, ' a_u_min: ', a_u_min ) ## set probe

        if up_u :
            theta = [ np.arccos(1- 2*a_u_max)/2/pi , np.arccos(1- 2*a_u_min)/2/pi     ]
            theta_u_min = min(theta)
            theta_u_max = max(theta)
            
        else :
            theta = [1 -  np.arccos(1- 2*a_u_max)/2/pi ,1 -  np.arccos(1- 2*a_u_min)/2/pi     ]
            theta_u_min = min(theta)
            theta_u_max = max(theta)

        # print('theta_u_max :', theta_u_max, 'theta_u_min :', theta_u_min ) ## set probe 

        ## calculate confidence interval
        theta_low = ( int(K_u*theta_low ) + theta_u_min )/K_u
        theta_up = ( int(K_u*theta_up ) + theta_u_max )/K_u
        a_low = (np.sin(2 * pi * theta_low))**2
        a_up = (np.sin(2 * pi * theta_up))**2


        ## set probe 
        # print('theta_low: ', theta_low, ' theta_up: ', theta_up )
        print('a_low: ', a_low, ' a_up: ', a_up )

        theta_lows.append(theta_low)
        theta_ups.append(theta_up)

    return (a_low + a_up)*0.5  




In [18]:
p = 0.75
IQAE_single_qubit(p)

a_u : 0.72
a_u_max:  0.846056610900578  a_u_min:  0.5939433890994219
a_low:  0.5939433890994219  a_up:  0.846056610900578
a_u : 0.77
a_u_max:  0.8960566109005781  a_u_min:  0.643943389099422
a_low:  0.643943389099422  a_up:  0.8960566109005781
a_u : 0.82
a_u_max:  0.946056610900578  a_u_min:  0.6939433890994219
a_low:  0.6939433890994217  a_up:  0.9460566109005779
a_u : 0.83
a_u_max:  0.956056610900578  a_u_min:  0.7039433890994219
a_low:  0.7039433890994218  a_up:  0.956056610900578
a_u : 0.7
a_u_max:  0.826056610900578  a_u_min:  0.5739433890994219
a_low:  0.5739433890994219  a_up:  0.826056610900578
a_u : 0.73
a_u_max:  0.856056610900578  a_u_min:  0.6039433890994219
a_low:  0.6039433890994219  a_up:  0.856056610900578
a_u : 0.7
a_u_max:  0.826056610900578  a_u_min:  0.5739433890994219
a_low:  0.5739433890994219  a_up:  0.826056610900578
a_u : 0.7
a_u_max:  0.826056610900578  a_u_min:  0.5739433890994219
a_low:  0.5739433890994219  a_up:  0.826056610900578
a_u : 0.66
a_u_max:  0.786

0.7533379836141394

In [22]:
theta_low, theta_ups

(0.017684837685487495,
 [0.02027523116026136,
  0.020350486172503863,
  0.02042316767856758,
  0.020498422690810078,
  0.02057281656465424,
  0.020648936554250086,
  0.02071906857396178,
  0.020791750080025498,
  0.0208576599824292,
  0.02092103776755038,
  0.02099802699447381,
  0.02106900679116099,
  0.021141688297224705,
  0.021213517802380566,
  0.02128280362286657,
  0.021354633128022427,
  0.021428169297041277,
  0.02149407919944498,
  0.021572821456303338,
  0.021646357625322188,
  0.021720751499166354,
  0.021802166932960606,
  0.021872298952672304,
  0.021943278749359484,
  0.02201171960044525,
  0.022081005420931256,
  0.02214860218901383,
  0.02221958198570101,
  0.02228549188810471,
  0.022349714406232023,
  0.02241984642594372,
  0.02249338259496257,
  0.022564362391649752,
  0.022636191896805612,
  0.022709728065824462,
  0.02278155757098032,
  0.02285681258322282,
  0.022926944602934516,
  0.022997924399621696,
  0.023078442269933202,
  0.023151123775996922,
  0.02321703

In [ ]:

IQAE_single_qubit(p, eps, alpha, Nshots ) , np.sqrt(p)

$$ theta $$


In [ ]:
np.mod(3*pi + 1 , pi )

1.0

In [ ]:
if True and True :
    print('1')

1
